In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline


import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler
import torch.nn.functional as F


In [100]:
train_idx = !ls data/composites/train/ 

In [101]:
val_idx = !ls data/composites/valid/

In [102]:
train_idx = [int(i.split('.')[0]) for i in train_idx]
val_idx = [int(i.split('.')[0]) for i in val_idx]

In [103]:
train_data = pd.read_json('./train/processed/train.json')
test_data = pd.read_json('./test/processed/test.json')

In [104]:
#create training data array
train_data_array = []
for i, row in train_data.iterrows():
    band_1 = row['band_1']
    band_2 = row['band_2']
    stacked = np.dstack([np.array(band_1).reshape(75, 75), np.array(band_2).reshape(75, 75),
                    np.array(band_1).reshape(75, 75) / np.array(band_2).reshape(75, 75),
                    (np.array(band_1).reshape(75, 75) + np.array(band_2).reshape(75, 75))/2])
    train_data_array.append(stacked)

train_data_array = np.array(train_data_array)

In [105]:
test_data_array = []
for i, row in test_data.iterrows():
    band_1 = row['band_1']
    band_2 = row['band_2']
    stacked = np.dstack([np.array(band_1).reshape(75, 75), np.array(band_2).reshape(75, 75),
                    np.array(band_1).reshape(75, 75) / np.array(band_2).reshape(75, 75),
                    (np.array(band_1).reshape(75, 75) + np.array(band_2).reshape(75, 75))/2])
    test_data_array.append(stacked)

test_data_array = np.array(test_data_array)

In [106]:
print(train_data_array.shape)
print(test_data_array.shape)

(1604, 75, 75, 4)
(8424, 75, 75, 4)


In [107]:
train_data_array = train_data_array.transpose((0, 3, 2, 1))
test_data_array = test_data_array.transpose((0, 3, 2, 1))

### CREATE PYTORCH DATASETS

In [108]:
# Creating FloatTensors from numpy arrays
train_data_tensor = torch.FloatTensor(train_data_array[train_idx, :, :, :])
valid_data_tensor = torch.FloatTensor(train_data_array[val_idx, :, :, :])
test_data_tensor = torch.FloatTensor(test_data_array)

In [126]:
# Creating LongTensor from responses of train, valid, test
train_y_tensor = torch.LongTensor(np.array(train_data.is_iceberg.loc[train_idx].values).astype('int64'))
valid_y_tensor = torch.LongTensor(np.array(train_data.is_iceberg.loc[val_idx].values).astype('int64'))
test_y_tensor = torch.LongTensor(np.zeros((test_data.shape[0], 1)).astype('int64'))

In [127]:
# Creating TensorDatasets
train_dataset = TensorDataset(data_tensor=train_data_tensor, target_tensor=train_y_tensor)
valid_dataset = TensorDataset(data_tensor=valid_data_tensor, target_tensor=valid_y_tensor)
test_dataset = TensorDataset(data_tensor=test_data_tensor, target_tensor=test_y_tensor)

In [128]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=15,
                                         shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(valid_dataset, batch_size=15,
                                         shuffle=True, num_workers=4)

In [129]:
dataloaders = {'train':trainloader, 'val':valloader}
datasizes = {'train':train_data_tensor.size(0), 'val':valid_data_tensor.size(0)}

### MODEL

In [130]:
def conv_calc(W, F, S, P): return (W - F + 2*P) / S + 1
def pool_calc(W, F, S): return (W - F) / S + 1

In [131]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=4,
                      kernel_size=4,
                      out_channels=64,
                      padding=1
            ), 
            nn.BatchNorm2d(64),
            nn.LeakyReLU()                   # try leak LeakyReLU later as well
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64,
                      kernel_size=3,
                      out_channels=64,
                      padding=1
            ),                 
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),                   # try leak LeakyReLU later as well
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25)
        )
        
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=64,
                      kernel_size=4,
                      out_channels=128,
                      padding=1
            ),              
            nn.BatchNorm2d(128),
            nn.LeakyReLU()                   # try leak LeakyReLU later as well
        )
        
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=128,
                      kernel_size=3,
                      out_channels=128,
                      padding=1
            ),             
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),                   # try leak LeakyReLU later as well
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25)   
        )
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=128,
                      kernel_size=3,
                      out_channels=256,
                      padding=1
            ),                   
            nn.BatchNorm2d(256),
            nn.LeakyReLU()                   # try leak LeakyReLU later as well
        )
        
        
        self.conv6 = nn.Sequential(
            nn.Conv2d(in_channels=256,
                      kernel_size=3,
                      out_channels=256,
                      padding=1
            ),                   
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),                   # try leak LeakyReLU later as well
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25)   
        )

        self.conv7 = nn.Sequential(
            nn.Conv2d(in_channels=256,
                      kernel_size=4,
                      out_channels=512,
                      padding=1
            ),            
            nn.BatchNorm2d(512),
            nn.LeakyReLU()                   # try leak LeakyReLU later as well
        )
        
        
        self.conv8 = nn.Sequential(
            nn.Conv2d(in_channels=512,
                      kernel_size=3,
                      out_channels=512,
                      padding=1
            ),
            nn.BatchNorm2d(512),           
            nn.LeakyReLU(),                   # try leak LeakyReLU later as well
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25)   
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(512*4*4, 2048),
            nn.BatchNorm1d(2048),
            nn.LeakyReLU(),
            nn.Dropout(p=0.5)
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Dropout(p=0.5)
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(1024, 2),
            nn.LeakyReLU(),
            nn.Dropout(p=0.5)
        )
    
    def forward(self, x):
        x = self.conv1(x) # output (64, 74, 74)
        x = self.conv2(x) # output (64, 37, 37)
        x = self.conv3(x) # output (128, 36, 36)
        x = self.conv4(x) # output (128, 18, 18)
        x = self.conv5(x) # output (256, 18, 18)
        x = self.conv6(x) # output (256, 9, 9)
        x = self.conv7(x) # output (512, 8, 8)
        x = self.conv8(x) # output (512, 4, 4)
        x = x.view(x.size(0), -1) # flatten the output 8192 nodes
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return F.log_softmax(x)

In [135]:
net = CNN()
optimizer = torch.optim.Adam(net.parameters(), lr=0.005)
loss_func = nn.CrossEntropyLoss()

In [136]:
net.parameters

<bound method Module.parameters of CNN (
  (conv1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.01)
  )
  (conv2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.01)
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (4): Dropout (p = 0.25)
  )
  (conv3): Sequential (
    (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.01)
  )
  (conv4): Sequential (
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.01)
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (4): Dropout (p = 0.25)
  )
 

In [137]:
n_cycles = 20

for epoch in range(n_cycles):
    for step, (x, y) in enumerate(trainloader):
        b_x = Variable(x)
        b_y = Variable(y)
        output = net(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch}')

Process Process-369:
Process Process-370:
Process Process-372:
Process Process-371:
Traceback (most recent call last):
  File "/home/kturgutlu/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/kturgutlu/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kturgutlu/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/kturgutlu/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kturgutlu/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/kturgutlu/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/uti

KeyboardInterrupt: 

KeyboardInterrupt
